### 원핫 인코딩의 한계

[관련링크](https://wikidocs.net/22647)

- 원-핫 벡터는 단어의 유사도를 표현하지 못한다는 단점이 있습니다. 예를 들어서 늑대, 호랑이, 강아지, 고양이라는 4개의 단어에 대해서 원-핫 인코딩을 해서 각각, [1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 1, 0], [0, 0, 0, 1]이라는 원-핫 벡터를 부여받았다고 합시다. 이때 원-핫 벡터로는 강아지와 늑대가 유사하고, 호랑이와 고양이가 유사하다는 것을 표현할 수가 없습니다. 좀 더 극단적으로는 강아지, 개, 냉장고라는 단어가 있을 때 강아지라는 단어가 개와 냉장고라는 단어 중 어떤 단어와 더 유사한지도 알 수 없습니다.

- 이러한 단점을 해결하기 위해 단어의 잠재 의미를 반영하여 다차원 공간에 벡터화 하는 기법으로 크게 두 가지가 있습니다. 첫째는 카운트 기반의 벡터화 방법인 LSA(잠재 의미 분석), HAL 등이 있으며, 둘째는 예측 기반으로 벡터화하는 NNLM, RNNLM, Word2Vec, FastText 등이 있습니다. 그리고 카운트 기반과 예측 기반 두 가지 방법을 모두 사용하는 방법으로 GloVe라는 방법이 존재합니다.

In [ ]:
# 그래도 원핫 인코딩 형태로 만들어나 보자...

In [ ]:
# from konlpy.tag import Mecab
from tqdm import tqdm
import re
import pickle
import csv
import pandas as pd
import numpy as np
from google.colab import drive

drive.mount('/content/gdrive/')

# 병명증상정보.csv : "병명/증상 정보 data"를 csv로 변환한 파일
df = pd.read_csv("/content/gdrive/MyDrive/멀티캠퍼스_DS_DE_33/2nd_pjt/병명증상정보.csv", encoding='cp949')
df.head()

Mounted at /content/gdrive/


,담당자,부위,증상,질환명,진료과,TAG
0,박장훈,복부,신생아의 98%는 출생 후 24시간 내에 짙은 녹색의 태변을 배설합니다. 아기의 항...,선천성 거대결장증,소아외과,"히르쉬스프룽병 ,선청성대장질환 ,거대결장 ,대장조영술검사"
1,박장훈,복부,간에서 만들어진 담즙은 담도를 통해 소장으로 분비돼 소화를 돕는 데 사용됩니다. 그...,담도폐쇄증,"소화기내과, 간담췌외과, 소아외과","담도, 담도폐쇄증 ,billiary atresia ,간경변증 ,간이식 ,담도조영술 ..."
2,박장훈,복부,"조기 간암의 증상으로 피로감, 전신 쇠약감, 소화 불량, 상복부에 느껴지는 뻐근함,...",간암,"간담췌외과, 방사선종양학과, 혈액종양내과, 소화기내과, 간암센터","피로감, 구토 ,복통 ,황달 ,메스꺼움 ,식욕부진 ,체중감소 ,AFP ..."
3,박장훈,복부,"대표적인 증상으로는 복통, 설사, 체중 감소를 들 수 있습니다. 특히 10~20대 ...",크론병 Crohn disease,"소화기내과, 소아소화기영양과","복통 ,설사 ,식욕부진 ,체중감소 ,치루 ,장폐색 국소성 장염 ,궤양성 ..."
4,박장훈,복부,비만은 체내 근육량에 비해 지방조직이 과다한 상태를 의미합니다. 대한비만학회에서는 ...,고도비만 Severe obesity,"내분비내과, 가정의학과, 내분비당뇨병센터, 비만대사수술센터","빈혈, 속 쓰림 ,오심, 구토 ,복통 ,체중감소 ,위암개복수술 ,위암합병증 ..."


In [ ]:
# n-gram 어휘 벡터 생성
def getNgramWord(N,txt):
    txt = txt.split()
    ngrams = [txt[i:i+N] for i in range(len(txt)-N+1)]
    return ngrams

In [ ]:
pip install konlpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 32.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 488.6/488.6 kB 36.6 MB/s eta 0:00:00


In [ ]:
import konlpy
from konlpy.tag import Kkma, Komoran, Hannanum, Okt
from konlpy.utils import pprint
from konlpy.tag import Mecab
import time

hannanum = Hannanum()
kkma = Kkma()
komoran = Komoran()
okt = Okt()

In [ ]:
# okt, 이전 챕터에서 실행시간이 가장 짧고 성능이 가장 좋다고 판단

columns = []

a = time.time()

for i in df['증상']:

  # 단어를 몇 개까지 이어 붙일지 결정
  for j in range(2,10):

    for k in range(len(getNgramWord(j,i))):
      # 정상 출력되는지 확인용
      # print(getNgramWord(j,i)[k], hannanum.nouns(str(getNgramWord(j,i)[k][-1])))

      change_word = okt.nouns(str(getNgramWord(j,i)[k][-1]))


      tmp2 = getNgramWord(j,i)[k][:j - 1] + change_word
      print(getNgramWord(j,i)[k])
      print(tmp2)
      print()
      if tmp2 != "":
        columns.append(' '.join(tmp2))

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
['탈모는', '머리카락이', '점진적으로', '가늘어지는', '것이', '주요']

['머리카락이', '점진적으로', '가늘어지는', '것이', '주요', '증상이므로']
['머리카락이', '점진적으로', '가늘어지는', '것이', '주요', '증상', '므']

['점진적으로', '가늘어지는', '것이', '주요', '증상이므로', '뒤통수처럼']
['점진적으로', '가늘어지는', '것이', '주요', '증상이므로', '뒤통수']

['가늘어지는', '것이', '주요', '증상이므로', '뒤통수처럼', '머리카락이']
['가늘어지는', '것이', '주요', '증상이므로', '뒤통수처럼', '머리카락']

['것이', '주요', '증상이므로', '뒤통수처럼', '머리카락이', '빠지지']
['것이', '주요', '증상이므로', '뒤통수처럼', '머리카락이']

['주요', '증상이므로', '뒤통수처럼', '머리카락이', '빠지지', '않는']
['주요', '증상이므로', '뒤통수처럼', '머리카락이', '빠지지']

['증상이므로', '뒤통수처럼', '머리카락이', '빠지지', '않는', '부위와']
['증상이므로', '뒤통수처럼', '머리카락이', '빠지지', '않는', '부위']

['뒤통수처럼', '머리카락이', '빠지지', '않는', '부위와', '비교해']
['뒤통수처럼', '머리카락이', '빠지지', '않는', '부위와', '비교']

['머리카락이', '빠지지', '않는', '부위와', '비교해', '정수리나']
['머리카락이', '빠지지', '않는', '부위와', '비교해', '정수리']

['빠지지', '않는', '부위와', '비교해', '정수리나', '앞머리의']
['빠지지', '않는', '부위와', '비교해', '정수리나', '앞머리']

['않는', '부위와', '비교해', '정수리나', '앞머리의', '머리카락의']
['않는', '부위와', '비교해', '정수리나', '앞

In [ ]:
print(columns)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
len(columns)

162221

In [ ]:
df

,담당자,부위,증상,질환명,진료과,TAG
0,박장훈,복부,신생아의 98%는 출생 후 24시간 내에 짙은 녹색의 태변을 배설합니다. 아기의 항...,선천성 거대결장증,소아외과,"히르쉬스프룽병 ,선청성대장질환 ,거대결장 ,대장조영술검사"
1,박장훈,복부,간에서 만들어진 담즙은 담도를 통해 소장으로 분비돼 소화를 돕는 데 사용됩니다. 그...,담도폐쇄증,"소화기내과, 간담췌외과, 소아외과","담도, 담도폐쇄증 ,billiary atresia ,간경변증 ,간이식 ,담도조영술 ..."
2,박장훈,복부,"조기 간암의 증상으로 피로감, 전신 쇠약감, 소화 불량, 상복부에 느껴지는 뻐근함,...",간암,"간담췌외과, 방사선종양학과, 혈액종양내과, 소화기내과, 간암센터","피로감, 구토 ,복통 ,황달 ,메스꺼움 ,식욕부진 ,체중감소 ,AFP ..."
3,박장훈,복부,"대표적인 증상으로는 복통, 설사, 체중 감소를 들 수 있습니다. 특히 10~20대 ...",크론병 Crohn disease,"소화기내과, 소아소화기영양과","복통 ,설사 ,식욕부진 ,체중감소 ,치루 ,장폐색 국소성 장염 ,궤양성 ..."
4,박장훈,복부,비만은 체내 근육량에 비해 지방조직이 과다한 상태를 의미합니다. 대한비만학회에서는 ...,고도비만 Severe obesity,"내분비내과, 가정의학과, 내분비당뇨병센터, 비만대사수술센터","빈혈, 속 쓰림 ,오심, 구토 ,복통 ,체중감소 ,위암개복수술 ,위암합병증 ..."
...,...,...,...,...,...,...
328,정의석,머리,"두개골이 완전히 닫히지 않은 2세 이하의 소아에게 수두증이 생기면, 뇌척수액이 축적...",수두증,신경외과,"큰 머리둘레, 대두증, 구토, 졸음, 두통, 발작, 눈이 아래로 처짐, 일몰 현상,..."
329,정의석,머리,시야가 어두워짐. 빛이 반짝거리거나 아지랑이가 피는 것 같은 시각 조짐. 두통. 전...,편두통,신경과,"박동성 두통, 관자놀이 통증, 오심, 구토, 어두운 시야, 아지랑이, 반짝거림, 조..."
330,정의석,머리,"발열, 피로감, 몸살 기운, 기침, 구토, 식욕 감소, 두통, 목이 뻣뻣함, 의식장...",바이러스 수막염,"감염내과, 신경과","발열, 피로, 목 뻣뻣함, 두통, 보챔, 기면, 졸음, 엔테로바이러스, 장바이러스,..."
331,정의석,머리,1~2주 잠복기. 대부분 무증상. 고열. 두통. 어지럼증. 구토. 설사. 의식 혼미...,일본뇌염,"감염내과, 신경과, 소아감염면역과, 소아신경과","고열, 두통, 복통, 설사, 경련, 모기, 사백신, 생백신"


In [ ]:
dff = pd.DataFrame(data = None, columns=columns, index = df['질환명'])

In [ ]:
dff
# 데이터 대략 5000만개

,신생아의,98%는 출생,출생 후,후,24시간 내,내에,짙은 녹색,녹색의 태변,태변을 배설,배설합니다. 아기,...,뒤 뻣뻣. 신경 마비. 의식장애. 경련,미열. 두통. 피로감. 목 뒤 뻣뻣. 신경,두통. 피로감. 목 뒤 뻣뻣. 신경 마비,피로감. 목 뒤 뻣뻣. 신경 마비. 의식 장애,목 뒤 뻣뻣. 신경 마비. 의식장애. 경련,미열. 두통. 피로감. 목 뒤 뻣뻣. 신경 마비,두통. 피로감. 목 뒤 뻣뻣. 신경 마비. 의식 장애,피로감. 목 뒤 뻣뻣. 신경 마비. 의식장애. 경련,미열. 두통. 피로감. 목 뒤 뻣뻣. 신경 마비. 의식 장애,두통. 피로감. 목 뒤 뻣뻣. 신경 마비. 의식장애. 경련
질환명,,,,,,,,,,,,,,,,,,,,,
선천성 거대결장증,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
담도폐쇄증,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
간암,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
크론병 Crohn disease,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
고도비만 Severe obesity,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
수두증,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
편두통,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
바이러스 수막염,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
for i in dff.index:   # 질환 명
  for j in dff.columns:   # N-gram 데이터칼럼
    if j in list(df[df['질환명'] == i]['증상']):
      dff[j][i] = 1
    # else:
# 예상소요 시간 6분 * 333 = 33.3시간
# ?

In [ ]:
import warnings
warnings.filterwarnings('ignore')

for i in dff.index:
  for j in dff.columns:
    if j in str(list(df[df['질환명'] == i]['증상'])[0]):
      dff[j][i] = 1
  # print(dff['증상'][i])

In [ ]:
dff.to_csv("test,csv")

# 데이터 저장하는것만 해도 시간 오래 걸린다... 2분 20초
# 파일 안에 거진 비어있는데 59MB...

In [ ]:
dff

,신생아의,98%는 출생,출생 후,후,24시간 내,내에,짙은 녹색,녹색의 태변,태변을 배설,배설합니다. 아기,...,뒤 뻣뻣. 신경 마비. 의식장애. 경련,미열. 두통. 피로감. 목 뒤 뻣뻣. 신경,두통. 피로감. 목 뒤 뻣뻣. 신경 마비,피로감. 목 뒤 뻣뻣. 신경 마비. 의식 장애,목 뒤 뻣뻣. 신경 마비. 의식장애. 경련,미열. 두통. 피로감. 목 뒤 뻣뻣. 신경 마비,두통. 피로감. 목 뒤 뻣뻣. 신경 마비. 의식 장애,피로감. 목 뒤 뻣뻣. 신경 마비. 의식장애. 경련,미열. 두통. 피로감. 목 뒤 뻣뻣. 신경 마비. 의식 장애,두통. 피로감. 목 뒤 뻣뻣. 신경 마비. 의식장애. 경련
질환명,,,,,,,,,,,,,,,,,,,,,
선천성 거대결장증,1,1,NaN,NaN,1,NaN,1,1,NaN,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
담도폐쇄증,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
간암,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
크론병 Crohn disease,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
고도비만 Severe obesity,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
수두증,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
편두통,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
바이러스 수막염,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
